# Import des modules nécessaires

In [1]:
    # Pour manipuler efficacement des tables de données dans Python
    import pandas as pd

    # Pour faire des requêtes GET et POST
    import requests
    
    # Pour gérer les date
    import datetime as dt

In [2]:
print(pd.__version__)

0.23.0


In [3]:
print(requests.__version__)

2.18.4


# Requêtage d'une API REST

Pour lancer une requête GET, c'est ultra-simple : on utilise `requests.get(url)`.

Si vous voulez glisser des paramètres avec la requête, utilisez l'argument optionnel `params`, qui accepte un simple dictionnaire.

In [4]:
r = requests.get("https://data.angers.fr/api/records/1.0/search/",
                 params={
                     'dataset': 'horaires-theoriques-et-arrets-du-reseau-irigo-gtfs',
                     'rows': 10
                 })

In [5]:
g = requests.get("https://data.angers.fr/api/records/1.0/search/",
                 params={
                     'dataset': 'bus-tram-position-tr',
                     'rows': 200
                 })

Regardons la nature de ce qui nous est renvoyé.

In [6]:
type(g)

requests.models.Response

C'est un objet de la classe `Response`. Si vous voulez connaitre toutes les méthodes attachées, faites `help(r)`

In [7]:
help(g)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

Pour tester que tout a bien fonctionnné, on va vérifié le statut.

In [8]:
g.raise_for_status()

Si ça dit rien c'est que c'est bon :) Dans le doute, regardons le statut...

In [9]:
g.status_code

200

Statut 200 = succès !!

Pour récupérer le contenu de la réponse, vous pouvez utiliser `r.text`.

Il y a aussi un décodeur JSON, si on sait que la réponse est dans ce format. Du coup, il parse le JSON et rend un dictionnaire !

In [10]:
g.json()

{'nhits': 106,
 'parameters': {'dataset': ['bus-tram-position-tr'],
  'timezone': 'UTC',
  'rows': 200,
  'format': 'json'},
 'records': [{'datasetid': 'bus-tram-position-tr',
   'recordid': '14c506f09a99d91c098950a3fe823121db098640',
   'fields': {'etat': 'HLPS',
    'type': 'OmniArtE6',
    'ecart': 0,
    'novh': '681',
    'dest': 'PAS EN SERVICE',
    'idparcours': 268721408,
    'cap': 95,
    'nomarret': 'SORTIE BUS CTB',
    'idvh': 268436137,
    'iddesserte': 268721409,
    'mnemoligne': '04',
    'y': 2281381,
    'nomligne': 'BEAUCOUZE <> ST BARTHELEMY',
    'mnemoarret': 'SDCTB',
    'idarret': 671973,
    'harret': '2018-06-19T06:53:55+00:00',
    'x': 381077,
    'sv': '0423',
    'coordonnees': [47.4956, -0.56967987],
    'idligne': 268435460,
    'numarret': 17220},
   'geometry': {'type': 'Point', 'coordinates': [-0.56967987, 47.4956]},
   'record_timestamp': '2018-06-19T06:57:17+00:00'},
  {'datasetid': 'bus-tram-position-tr',
   'recordid': 'd2707eca39de09f79bcef7be

# Remplissage de DataFrame

On peut construire une `DataFrame` directement à partir d'un "dictionnaires de colonnes".

In [11]:
df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [3, 4, 1]
})

df

,a,b
0,1,3
1,2,4
2,3,1


Par exemple, listons les arrêts, leur id, leur nom et leurs coordonnées GPS.

In [12]:
dd = g.json()['records']

In [13]:
dd[0]

{'datasetid': 'bus-tram-position-tr',
 'recordid': '14c506f09a99d91c098950a3fe823121db098640',
 'fields': {'etat': 'HLPS',
  'type': 'OmniArtE6',
  'ecart': 0,
  'novh': '681',
  'dest': 'PAS EN SERVICE',
  'idparcours': 268721408,
  'cap': 95,
  'nomarret': 'SORTIE BUS CTB',
  'idvh': 268436137,
  'iddesserte': 268721409,
  'mnemoligne': '04',
  'y': 2281381,
  'nomligne': 'BEAUCOUZE <> ST BARTHELEMY',
  'mnemoarret': 'SDCTB',
  'idarret': 671973,
  'harret': '2018-06-19T06:53:55+00:00',
  'x': 381077,
  'sv': '0423',
  'coordonnees': [47.4956, -0.56967987],
  'idligne': 268435460,
  'numarret': 17220},
 'geometry': {'type': 'Point', 'coordinates': [-0.56967987, 47.4956]},
 'record_timestamp': '2018-06-19T06:57:17+00:00'}

In [14]:
#Arret
id_arret = [elem['fields']['idarret'] for elem in dd]
nom_arret = [elem['fields']['nomarret'] for elem in dd]
mne_arret = [elem['fields']['mnemoarret'] for elem in dd]

arret = pd.DataFrame({
    'id_arret': id_arret,
    'nom_arret': nom_arret,
    'mne_arret': mne_arret
})

arret.info()

arret.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 3 columns):
id_arret     106 non-null int64
nom_arret    106 non-null object
mne_arret    106 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ KB


,id_arret,nom_arret,mne_arret
94,671253,CARNOT Supprimé,CARNOT-E
20,672772,MARIUS BRIANT,MBRIAN-E
21,671059,RALLIEMENT,1RALL
12,671269,CEZANNE,CEZAN-E
88,671067,BAMAKO,2BAMA
7,671263,CENTRE COMMERCIAL,CCOMME-E
34,671977,GARE SEMARD,SEMARD
55,671582,JEAN BOUIN,JEBOUIN
48,671697,MADELEINE,MADELEIN
27,672581,AQUA VITA,AQVI-D5B


In [15]:
#Bus
id_vh = [elem['fields']['idvh'] for elem in dd]
type_vh = [elem['fields']['type'] for elem in dd]
etat_vh = [elem['fields']['etat'] for elem in dd]

bus = pd.DataFrame({
    'id_vh': id_vh,
    'type_vh': type_vh,
    'etat_vh': etat_vh
})

bus.info()

bus.sample(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 3 columns):
id_vh      106 non-null int64
type_vh    106 non-null object
etat_vh    106 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ KB


,id_vh,type_vh,etat_vh
26,268436466,CITADIS,TDEP
55,268436257,OmniSTDE6,LIGN
67,268435900,OMNICITY,LIGN
3,268436120,OMNIART,HLPS
14,268436079,MAN City,LIGN
69,268435911,OMNICITY,DEVP
81,268436123,OMNIART,LIGN
64,268436077,MAN City,TDEP
21,268436461,CITADIS,LIGN
16,268436110,OMNIART,LIGN


In [16]:
#Ligne
id_ligne = [elem['fields']['idligne'] for elem in dd]
nom_ligne = [elem['fields']['nomligne'] for elem in dd]
mne_ligne = [elem['fields']['mnemoligne'] for elem in dd]

ligne = pd.DataFrame({
    'id_ligne': id_arret,
    'nom_ligne': nom_ligne,
    'mne_ligne': mne_ligne
})

ligne.info()

ligne

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 3 columns):
id_ligne     106 non-null int64
nom_ligne    106 non-null object
mne_ligne    106 non-null object
dtypes: int64(1), object(2)
memory usage: 2.6+ KB


,id_ligne,nom_ligne,mne_ligne
0,671973,BEAUCOUZE <> ST BARTHELEMY,04
1,671375,BEAUCOUZE <> ST BARTHELEMY,04
2,671375,BOUCHEMAINE <> Z I EST,06
3,671973,BELLE BEILLE <> MONPLAISIR,01
4,671157,M-MARCILLE <> ST AUBIN LA SALLE,12
5,672636,Piscine MONPLAISIR,535
6,671593,M-MARCILLE <> ST AUBIN LA SALLE,12
7,671263,PONTS CE <> AQUAVITA H. RECULEE,08
8,671507,CIRCULAIRE VERNEAU GARE EUROPE,05
9,672042,ST LEZIN SORGES <> SCHWEITZER,10


In [17]:
#Trajet
id_bus = [elem['fields']['idvh'] for elem in dd]
id_ligne = [elem['fields']['idligne'] for elem in dd]
latitude = [elem['fields']['coordonnees'][0] for elem in dd]
longitude = [elem['fields']['coordonnees'][1] for elem in dd]

trajet = pd.DataFrame({
    'id_bus': id_bus,
    'id_ligne': id_ligne,
    'latitude': latitude,
    'longitude': longitude
})

trajet.info()

trajet

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 4 columns):
id_bus       106 non-null int64
id_ligne     106 non-null int64
latitude     106 non-null float64
longitude    106 non-null float64
dtypes: float64(2), int64(2)
memory usage: 3.4 KB


,id_bus,id_ligne,latitude,longitude
0,268436137,268435460,47.495600,-0.569680
1,268436259,268435460,47.476323,-0.492809
2,268435899,268435462,47.475119,-0.492731
3,268436120,268435457,47.495681,-0.569286
4,268435905,268435468,47.489367,-0.503753
5,268435879,268435515,47.486526,-0.521209
6,268435909,268435468,47.469206,-0.540589
7,268435910,268435464,47.437790,-0.526913
8,268435913,268435461,47.474968,-0.524890
9,268435994,268435466,47.483529,-0.528484


In [19]:
#Etape
id_arret = [elem['fields']['idarret'] for elem in dd]
harret = [elem['fields']['harret'] for elem in dd]
record_timestamp = [elem['record_timestamp'] for elem in dd]
ecart = [elem['fields']['ecart'] for elem in dd]

def transfo(row):
    return record_timestamp + dt.timedelta(seconds=ecart)

heure_estime = df.apply(transfo, axis='columns')

etape = pd.DataFrame({
    'id_arret': id_arret,
    'heure_theorique': harret,
    'heure_estime': heure_estime,
    'ecart': ecart
})

etape.info()

etape

TypeError: ('unsupported type for timedelta seconds component: list', 'occurred at index 0')